In [1]:
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import warnings
import random
import math

warnings.filterwarnings('ignore')

def index_of_agreement (s, o):
    s,o = filter_nan(s,o)
    ia = 1 -(np.sum((o-s)**2))/(np.sum((np.abs(s-np.mean(o))+np.abs(o-np.mean(o)))**2))
    return ia
def filter_nan(s,o):

    s = np.array(s.copy())
    o = np.array(o.copy())
    data = np.array([s.flatten(),o.flatten()])
    data = np.transpose(data)
    data = data[~np.isnan(data).any(1)]
    s = data[:,0]
    o = data[:,1]
    return s, o

In [2]:
data = pd.read_csv("final_dataset.csv", index_col=0)
data.index = pd.to_datetime(data.index)

FEATURES = ['Power', 'Temp','T_lag_1', 'T_lag_2', 'T_lag_3', 'E_lag_1', 'E_lag_2', 'E_lag_3','t_cos','t_sin', 'd_cos', 'd_sin', 'w_cos',  'w_sin', 'm_cos', 'm_sin' ]

data_test_1 = data[(data['month'] == 3) | (data['month'] == 4) | (data['month'] == 5)]
data_train_1 = data[(data['month'] != 3) & (data['month'] != 4) & (data['month'] != 5)]
data_test_1 = data_test_1[FEATURES]
data_train_1 = data_train_1[FEATURES]

data_test_2 = data[(data['month'] == 6) | (data['month'] == 7) | (data['month'] == 8)]
data_train_2 = data[(data['month'] != 6) & (data['month'] != 7) & (data['month'] != 8)]   
data_test_2 = data_test_2[FEATURES]
data_train_2 = data_train_2[FEATURES]

data_test_3 = data[(data['month'] == 9) | (data['month'] == 10) | (data['month'] == 11)]
data_train_3 = data[(data['month'] != 9) & (data['month'] != 10) & (data['month'] != 11)]  
data_test_3 = data_test_3[FEATURES]
data_train_3 = data_train_3[FEATURES]

data_test_4 = data[(data['month'] == 12) | (data['month'] == 1) | (data['month'] == 2)]
data_train_4 = data[(data['month'] != 12) & (data['month'] != 1) & (data['month'] != 2)]
data_test_4 = data_test_4[FEATURES]
data_train_4 = data_train_4[FEATURES]

In [3]:
def calc_criterions (params,n1,n2):

    features = ['Temp','T_lag_1', 'T_lag_2', 'T_lag_3', 'E_lag_1', 'E_lag_2', 'E_lag_3','t_cos','t_sin', 'd_cos', 'd_sin', 'w_cos',  'w_sin', 'm_cos', 'm_sin']
    target = ['Power']
    feature_variables = params[n1:]
    new_features = []
    number_of_ones = 0
    
    for i in range(0, len(feature_variables)):
        if (feature_variables[i] == 1):
            new_features += [features[i]]
    
    epochs_ = int(params[0])
    layers_ = int(params[1])
    neurons_ = int(params[2])
    batch_size_ = int(params[3])
    learning_rate_ = float(params[4])
    
    
    global_train_MAE = 0.0
    global_val_MAE = 0.0
    global_test_MAE = 0.0
    
    global_train_MSE = 0.0
    global_val_MSE = 0.0
    global_test_MSE = 0.0
    
    global_train_IA = 0.0
    global_val_IA = 0.0
    global_test_IA = 0.0
    
    global_train_R2 = 0.0
    global_val_R2 = 0.0
    global_test_R2 = 0.0  
    
    for part in [1,2,3,4]:
        data = pd.read_csv('final_dataset.csv', sep = ',', low_memory=False,index_col=0)
        if (part == 1):
            data_test = data[(data['month'] == 3) | (data['month'] == 4) | (data['month'] == 5)]
            data_train = data[(data['month'] != 3) & (data['month'] != 4) & (data['month'] != 5)]

        if (part == 2):
            data_test = data[(data['month'] == 6) | (data['month'] == 7) | (data['month'] == 8)]
            data_train = data[(data['month'] != 6) & (data['month'] != 7) & (data['month'] != 8)]   

        if (part == 3):
            data_test = data[(data['month'] == 9) | (data['month'] == 10) | (data['month'] == 11)]
            data_train = data[(data['month'] != 9) & (data['month'] != 10) & (data['month'] != 11)]  

        if (part == 4):
            data_test = data[(data['month'] == 12) | (data['month'] == 1) | (data['month'] == 2)]
            data_train = data[(data['month'] != 12) & (data['month'] != 1) & (data['month'] != 2)]        

        X_train = data_train[new_features]
        Y_train = data_train[target]
                                                                              
        X_test = data_test[new_features]
        Y_test = data_test[target]

        scaler = StandardScaler()

        scaler.fit(X_train)

        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        k=5
        X = X_train.copy()
        y = np.array(Y_train).flatten().copy()
        kf = KFold(n_splits=k, random_state=None, shuffle=False)
        kf.get_n_splits(X)

        validation_MAE = 0.0
        validation_MSE = 0.0
        validation_R2 = 0.0
        validation_IA = 0.0

        for train_index, test_index in kf.split(X):

            x_train, x_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            hidden_layer_sizes = []
            for i in range(0,layers_):
                hidden_layer_sizes.append(neurons_)
            hidden_layer_sizes = tuple(hidden_layer_sizes)
            mlp_regressor = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, max_iter=epochs_, solver='adam',
                                     batch_size=batch_size_,
                                     learning_rate_init=learning_rate_,learning_rate='constant')
            mlp_regressor.fit(x_train, y_train)
            y_pred = mlp_regressor.predict(x_test)
            
            validation_MAE+=mean_absolute_error(y_test, y_pred)
            validation_MSE+=mean_squared_error(y_test, y_pred, squared=False)
            validation_R2+=r2_score(y_test, y_pred)
            validation_IA+=index_of_agreement(y_test, y_pred)

        global_val_MAE += validation_MAE/k
        global_val_MSE += validation_MSE/k
        global_val_R2 += validation_R2/k
        global_val_IA += validation_IA/k

        hidden_layer_sizes = []
        for i in range(0,layers_):
            hidden_layer_sizes.append(neurons_)
        hidden_layer_sizes = tuple(hidden_layer_sizes)

        mlp_regressor = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, max_iter=epochs_, solver='adam',
                                     batch_size=batch_size_,
                                     learning_rate_init=learning_rate_,learning_rate='constant')
        mlp_regressor.fit(X_train, Y_train)
        Y_pred = mlp_regressor.predict(X_train)
            
        
        global_train_MAE += mean_absolute_error(Y_train, Y_pred)
        global_train_MSE += mean_squared_error(Y_train, Y_pred)
        global_train_R2 += r2_score(Y_train, Y_pred)
        global_train_IA += index_of_agreement(Y_train, Y_pred)

        Y_pred = mlp_regressor.predict(X_test)
        
        global_test_MAE+= mean_absolute_error(Y_test, Y_pred)
        global_test_MSE+= mean_squared_error(Y_test, Y_pred)
        global_test_R2+=r2_score(Y_test, Y_pred)
        global_test_IA+=index_of_agreement(Y_test, Y_pred)
        
   
    global_train_MAE = global_train_MAE/4
    global_val_MAE = global_val_MAE/4
    global_test_MAE = global_test_MAE/4
        
    
    global_train_MSE = global_train_MSE/4
    global_val_MSE = global_val_MSE/4
    global_test_MSE = global_test_MSE/4
    
    global_train_R2 = global_train_R2/4
    global_val_R2 = global_val_R2/4
    global_test_R2 = global_test_R2/4
    
    global_train_IA = global_train_IA/4
    global_val_IA = global_val_IA/4
    global_test_IA = global_test_IA/4

    return global_train_MAE, global_val_MAE, global_test_MAE,global_train_MSE, global_val_MSE, global_test_MSE, global_train_R2, global_val_R2, global_test_R2, global_train_IA, global_val_IA, global_test_IA


In [4]:
def fitness_function(data_test_1,data_train_1, data_test_2,data_train_2,data_test_3,data_train_3,data_test_4,data_train_4, x, variable_type, n1, n2, goal_number_of_features):
   
    FEATURES = ['Temp','T_lag_1', 'T_lag_2', 'T_lag_3', 'E_lag_1', 'E_lag_2', 'E_lag_3','t_cos','t_sin', 'd_cos', 'd_sin', 'w_cos',  'w_sin', 'm_cos', 'm_sin']
    TARGET = ['Power']
    
    params = x.copy()
    feature_variables = params[n1:]
    new_features = []
    number_of_ones = 0
    
    for i in range(0, len(feature_variables)):
        if (feature_variables[i] == 1):
            new_features += [FEATURES[i]]
            number_of_ones = number_of_ones +1
    for i in range(0, n1):
        
        if (variable_type[i] == 'int'):
            params[i] = int(params[i])
   
    epochs_ = int(params[0])
    layers_ = int(params[1])
    neurons_ = int(params[2])
    batch_size_ = int(params[3])
    learning_rate_ = float(params[4])
    
    global_val_MAE = 0.0
      
    
    for part in [1,2,3,4]:
        if (part == 1):
            data_test = data_test_1         
            data_train = data_train_1

        if (part == 2):
            data_test = data_test_2
            data_train = data_train_2

        if (part == 3):
            data_test = data_test_3
            data_train = data_train_3

        if (part == 4):
            data_test = data_test_4
            data_train = data_train_4

        target = ['Power']
        
        X_train = data_train[new_features]
        Y_train = data_train[target]
        
        X_test = data_test[new_features]
        Y_test = data_test[target]

        # Create a scaler object
        scaler = StandardScaler()

        # Fit the scaler on the train data
        scaler.fit(X_train)

        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        k=5
        X = X_train
        y = np.array(Y_train).flatten()
        kf = KFold(n_splits=k, random_state=None, shuffle=False)
        kf.get_n_splits(X)

        validation_MAE_in_part = 0.0

        
        for train_index, test_index in kf.split(X):

            x_train, x_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            hidden_layer_sizes = []
            for i in range(0,layers_):
                hidden_layer_sizes.append(neurons_)
            hidden_layer_sizes = tuple(hidden_layer_sizes)

            mlp_regressor = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, max_iter=epochs_, solver='adam',
                                     batch_size=batch_size_,
                                     learning_rate_init=learning_rate_,learning_rate='constant')
            
            mlp_regressor.fit(x_train, y_train)
            y_pred = mlp_regressor.predict(x_test)

            score = mean_absolute_error(y_test, y_pred)
            
            validation_MAE_in_part += score

        
        global_val_MAE += validation_MAE_in_part/k
    
    global_val_MAE =global_val_MAE/4
    
    penalty = abs(goal_number_of_features -number_of_ones ) +1 ##

    return global_val_MAE*penalty

In [5]:
def init_population(population, pop_size, N, a, b, variable_type):
    for i in range(0, pop_size):
        for j in range (0, N):
            if (variable_type[j] == 'real'):
                population[i][j] = random.random()*(b[j]-a[j])+a[j]
            if (variable_type[j] == 'int'):
                population[i][j] = int(random.random()*(b[j]-a[j])+a[j])
            if (variable_type[j] == 'bool'):
                if (random.random()>0.5):
                    population[i][j] = 1
                else:
                    population[i][j] = 0

def generate_indices(pop_size, A, p, i):
    r1=int(random.random()*pop_size)
    r2=int(random.random()*(pop_size+A))
    max_best_number = int(p[i]*pop_size)
    bests_fitness = np.argsort(fitness)[:max_best_number]
    pbest = np.random.choice(bests_fitness)
    while (r1 == r2 or i == r1 or i == r2 ):
        r1=int(random.random()*pop_size)
        r2=int(random.random()*(pop_size+A))

    return r1, r2, pbest

def borders (v, population, N, a, b, variable_type):
    for i in range(0, pop_size):
        for j in range (0, N):
            if (variable_type[j] == 'real'):
                if (v[i][j] > b[j]):
                    v[i][j] = (b[j]+population[i][j])/2
                if (v[i][j] < a[j]):
                    v[i][j] = (a[j]+population[i][j])/2
            if (variable_type[j] == 'int'):
                if (v[i][j] > b[j]):
                    v[i][j] = int((b[j]+population[i][j])/2)
                if (v[i][j] < a[j]):
                    v[i][j] = int((a[j]+population[i][j])/2)
def isNaN(num):
    return num != num

In [ ]:
n1 = 5
n2 = 15
N = n1 + n2
pop_size = 50

for goal_number_of_features in [1,2,3,5,7,10,15]:
    RUNS = 15
    columns = ['epochs', "layers", "neurons","batch_size","learning_rate", 'Temp','T_lag_1', 'T_lag_2', 'T_lag_3', 'E_lag_1', 'E_lag_2', 'E_lag_3','t_cos','t_sin', 'd_cos', 'd_sin', 'w_cos',  'w_sin', 'm_cos', 'm_sin',
            'MAE_train', 'MSE_train', 'IA_train', 'R2_train',
            'MAE_val',   'MSE_val',   'IA_val',   'R2_val',
            'MAE_test',  'MSE_test',  'IA_test',  'R2_test'
              ]
    df_stats = pd.DataFrame(columns = columns, index = range(RUNS))
    df_fitness = pd.DataFrame(columns = columns, index = range(RUNS))
    variable_type =  ['int','int','int','int','real',
                      'bool','bool','bool','bool','bool',
                      'bool','bool','bool','bool','bool',
                      'bool','bool','bool','bool','bool']

    a = [1, 2, 5,  1,  0.0001]
    b = [200,5, 50, 1024, 1.0]

    archive_size = pop_size
    H = 10

    best_in_RUN = []
    fitness_stat = pd.DataFrame()
    solution_stat = []

    while (RUNS>0):

        global_best = 1e300
        population = np.empty((pop_size,N))
        v = np.empty((pop_size,N))
        archive = np.empty((archive_size,N))
        fitness = np.empty(pop_size)
        fitness_new = np.empty(pop_size)
        F_history = np.empty(H)
        CR_history = np.empty(H)
        S_CR = []
        S_F = []
        w = []
        r = np.empty(pop_size)
        CR = np.empty(pop_size)
        F = np.empty(pop_size)
        p = np.empty(pop_size)
        FEV = 2000
        kratnost = FEV/100
        tracking_fitness = []
        tracking_solution = []
        init_population(population, pop_size, N, a, b, variable_type)

        for i in range (0, pop_size):

            x = population[i][:]
            solution = x.copy()
            sol = solution.copy()
            fit = fitness_function(data_test_1,data_train_1, data_test_2,data_train_2,data_test_3,data_train_3,data_test_4,data_train_4,solution, variable_type ,n1, n2, goal_number_of_features)
            fitness[i] = fit
            fitness_new[i] = fitness[i]
            FEV =FEV - 1

            if (fitness[i]<global_best):
                global_best = fitness[i]
                best_solution = solution.copy()

            if (FEV % kratnost == 0):
                tracking_fitness.append(global_best)
                tracking_solution = (best_solution)

        F_history[:] = 0.5
        CR_history[:] = 0.5
        A = 0
        k=0
        pmin = 5.0/pop_size

        while (FEV>0):
            CR_df = pd.DataFrame(pd.DataFrame(CR_history).T)

            S_CR = np.empty(pop_size)
            S_F = np.empty(pop_size)
            v[:][:]=population[:][:]
            for i in range (0, pop_size):
                r[i] = int(random.random()*H)
                CR[i] = np.random.normal(CR_history[int(r[i])], 0.1)
                if CR[i]>1:
                    CR[i] = 1
                if CR[i]<0:
                    CR[i] = 0

                F[i] = F_history[int(r[i])]+np.random.standard_cauchy()*0.1
                if F[i]>1:
                    F[i] = 1
                while( F[i]<0):
                    F[i] = F_history[int(r[i])]+np.random.standard_cauchy()*0.1
                p[i] = random.random()*(0.2-pmin)+pmin

                r1, r2, pbest = generate_indices(pop_size, A, p, i)

                j_rand = int(random.random()*N)
                for j in range (0,n1):
                    if (random.random()<CR[i] or j == j_rand):
                        if (r2 < pop_size):
                            v[i][j] = population[i][j]+F[i]*(population[pbest][j]-population[i][j])+F[i]*(population[r1][j]-population[r2][j])                       
                        if (r2 >= pop_size):
                            r2 = r2-pop_size
                            v[i][j] = population[i][j]+F[i]*(population[pbest][j]-population[i][j])+F[i]*(population[r1][j]-archive[r2][j])

                for j in range (n1,n1 + n2):

                    if (random.random()<CR[i] or j == j_rand):
                        rnd_bool_var = pop_size+1
                        var_list = [i,pbest,r1,r2]
                        while (rnd_bool_var >= pop_size):  
                            rnd_bool_var = random.choice(var_list)

                        v[i][j] = population[rnd_bool_var][j]


                if (random.random()<(1.0/(n2))):
                    rnd_ind =random.randint(n1, n1 + n2-1) 
                    if (v[i][rnd_ind] == 0):
                        v[i][rnd_ind] =1
                    else:
                         v[i][rnd_ind] =1

                for j in range (0,n1):
                    if ( (isNaN(v[i][j]) == 1)):
                        if variable_type[j] == 'real':
                            v[i][j] = random.uniform(a[j], b[j])
                        if variable_type[j] == 'int':
                            v[i][j] = random.randint(int(a[j]), int(b[j]))

                check_sum_ones = 0
                for j in range (n1,n1 + n2):
                    if (v[i][j] == 1):
                        check_sum_ones = check_sum_ones +1
                if (check_sum_ones == 0):
                    v[i][random.randint(n1, n1 + n2-1)] = 1
                borders (v, population, N, a, b, variable_type)

            w = np.array([])
            S_CR = np.array([])
            S_F = np.array([])
            for i in range (0, pop_size):

                solution = v[i][:]
                x = solution = v[i][:]
                fit = fitness_function(data_test_1,data_train_1, data_test_2,data_train_2,data_test_3,data_train_3,data_test_4,data_train_4,x, variable_type ,n1, n2, goal_number_of_features)
                tf.keras.backend.clear_session()
                fitness_new[i] = fit
                FEV = FEV-1
                if (fitness_new[i]<fitness[i]):
                    S_CR = np.append(S_CR, CR[i])
                    S_F = np.append(S_F, F[i])
                    w = np.append(w,(fitness[i]-fitness_new[i]))
                    if (A<archive_size):
                        archive[i] = population[i][:]
                        A=A+1
                    if (A >= archive_size):
                        rnd_index = int(random.random()*archive_size)
                        archive[rnd_index] = population[i][:]

                    population[i][:] = solution

                    fitness[i] =fitness_new[i]

                    if (fitness[i]<global_best):
                        global_best = fitness[i]
                        best_solution = solution.copy()

                if (FEV % kratnost == 0):
                    tracking_fitness.append(global_best)
                    tracking_solution = (best_solution)

            total_w = np.sum(w)
            w = w/total_w

            new_CR = np.sum(w*S_F)
            new_F = np.sum(w*S_F*S_F)/np.sum(w*S_F)
            if (new_CR >0 and new_F>0):
                F_history[k]=new_F
                CR_history[k]=new_CR
                k=k+1
                if (k>H-1):
                    k=0

        RUNS = RUNS - 1
        tracking_fitness = pd.DataFrame(tracking_fitness)

        fitness_stat = pd.concat([fitness_stat,tracking_fitness], axis=1)
        solution_stat.append(tracking_solution)

        best_in_RUN.append(global_best)
        best_solution[0] = int(best_solution[0])
        best_solution[1] = int(best_solution[1])
        best_solution[2] = int(best_solution[2])
        best_solution[3] = int(best_solution[3])
        df_stats.iloc[RUNS,:len(best_solution)] = best_solution#########################
        criberions = calc_criterions(best_solution,n1,n2)

        df_stats.iloc[RUNS]['MAE_train'] = criberions[0]
        df_stats.iloc[RUNS]['MAE_val'] = criberions[1]
        df_stats.iloc[RUNS]['MAE_test'] = criberions[2]

        df_stats.iloc[RUNS]['MSE_train'] = criberions[3]
        df_stats.iloc[RUNS]['MSE_val'] = criberions[4]
        df_stats.iloc[RUNS]['MSE_test'] = criberions[5]

        df_stats.iloc[RUNS]['R2_train'] = criberions[6]
        df_stats.iloc[RUNS]['R2_val'] = criberions[7]
        df_stats.iloc[RUNS]['R2_test'] = criberions[8]

        df_stats.iloc[RUNS]['IA_train'] = criberions[9]
        df_stats.iloc[RUNS]['IA_val'] = criberions[10]
        df_stats.iloc[RUNS]['IA_test'] = criberions[11]

    file_name = 'NN_' + str(goal_number_of_features)+'.csv'
    df_stats.to_csv('NN_' + str(goal_number_of_features)+'.csv')
    fitness_stat.to_csv('NN_fitness' + str(goal_number_of_features)+'.csv')
    test = solution_stat.copy()
    test.append(best_in_RUN)
    test = pd.DataFrame(test)